In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
                    .appName("WordCount")\
                    .getOrCreate()

In [3]:
spark.version

u'2.0.2'

In [4]:
lines = spark.read.text('README.md').rdd.map(lambda r: r[0])

In [5]:
import re

In [6]:
def remove_punc(line):
    line = line.lower().strip()
    line = re.sub('[^0-9a-zA-Z ]', '', line)
    return line

In [7]:
counts = lines.map(remove_punc)\
             .flatMap(lambda line: line.split(' '))\
             .filter(lambda word: word != '')\
             .map(lambda word: (word, 1))\
             .reduceByKey(lambda x, y: x + y)

In [8]:
top20counts = counts.takeOrdered(20, lambda x: -x[1])

In [9]:
for item in top20counts:
    print item[0], item[1]

the 23
spark 17
to 16
for 14
and 12
run 9
you 9
a 9
can 7
using 6
is 6
in 6
on 5
of 5
example 5
building 5
hadoop 5
documentation 5
python 4
examples 4


In [10]:
spark.stop()